In [1]:
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('795813_1365687_bundle_archive/input_bcell.csv')

dataset.head()

,parent_protein_id,protein_seq,start_position,end_position,peptide_seq,chou_fasman,emini,kolaskar_tongaonkar,parker,isoelectric_point,aromaticity,hydrophobicity,stability,target
0,A2T3T0,MDVLYSLSKTLKDARDKIVEGTLYSNVSDLIQQFNQMIITMNGNEF...,161,165,SASFT,1.016,0.703,1.018,2.22,5.810364,0.103275,-0.143829,40.273300,1
1,F0V2I4,MTIHKVAINGFGRIGRLLFRNLLSSQGVQVVAVNDVVDIKVLTHLL...,251,255,LCLKI,0.770,0.179,1.199,-3.86,6.210876,0.065476,-0.036905,24.998512,1
2,O75508,MVATCLQVVGFVTSFVGWIGVIVTTSTNDWVVTCGYTIPTCRKLDE...,145,149,AHRET,0.852,3.427,0.960,4.28,8.223938,0.091787,0.879227,27.863333,1
3,O84462,MTNSISGYQPTVTTSTSSTTSASGASGSLGASSVSTTANATVTQTA...,152,156,SNYDD,1.410,2.548,0.936,6.32,4.237976,0.044776,-0.521393,30.765373,1
4,P00918,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,85,89,DGTYR,1.214,1.908,0.937,4.64,6.867493,0.103846,-0.578846,21.684615,1


In [2]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.05)

In [3]:
from pycaret.classification import *

experiment = setup(
    data = train 
    ,target = 'target'
    ,ignore_features = ['parent_protein_id', 'protein_seq', 'peptide_seq']
    ,normalize = True
)

Setup Succesfully Completed!


,Description,Value
0,session_id,7158
1,Target Type,Binary
2,Label Encoded,"0: 0, 1: 1"
3,Original Data,"(13667, 14)"
4,Missing Values,False
5,Numeric Features,10
6,Categorical Features,3
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [4]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Extra Trees Classifier,0.8511,0.8947,0.6459,0.7683,0.7015,0.6033,0.6076,0.3788
1,CatBoost Classifier,0.8408,0.8868,0.5865,0.7709,0.6659,0.5639,0.5732,6.2583
2,Extreme Gradient Boosting,0.8407,0.8843,0.6089,0.7559,0.6739,0.5702,0.5764,0.4342
3,Random Forest Classifier,0.8387,0.8765,0.5799,0.7684,0.6606,0.5575,0.5673,0.1237
4,Light Gradient Boosting Machine,0.8369,0.8859,0.5830,0.7592,0.6594,0.5546,0.5631,0.1372
5,Gradient Boosting Classifier,0.8041,0.8529,0.4239,0.7417,0.5392,0.4265,0.4535,1.4217
6,Decision Tree Classifier,0.7997,0.7403,0.6108,0.6357,0.6227,0.4865,0.4869,0.0690
7,K Neighbors Classifier,0.7842,0.7878,0.4394,0.6515,0.5238,0.3913,0.4045,0.0247
8,Ada Boost Classifier,0.7730,0.8043,0.3440,0.6550,0.4503,0.3245,0.3516,0.3629
9,Logistic Regression,0.7352,0.6551,0.0587,0.6179,0.1066,0.0628,0.1258,0.0300


ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=7158, verbose=0,
                     warm_start=False)

In [5]:
sars = pd.read_csv('795813_1365687_bundle_archive/input_sars.csv')

sars.head()

,parent_protein_id,protein_seq,start_position,end_position,peptide_seq,chou_fasman,emini,kolaskar_tongaonkar,parker,isoelectric_point,aromaticity,hydrophobicity,stability,target
0,AAU93319,MFIFLLFLTLTSGSDLDRCTTFDDVQAPNYTQHTSSMRGVYYPDEI...,1,17,MFIFLLFLTLTSGSDLD,0.887,0.040,1.056,-2.159,5.569763,0.116335,-0.061116,33.205116,0
1,AAU93319,MFIFLLFLTLTSGSDLDRCTTFDDVQAPNYTQHTSSMRGVYYPDEI...,1,15,MFIFLLFLTLTSGSD,0.869,0.047,1.056,-2.500,5.569763,0.116335,-0.061116,33.205116,0
2,AAU93319,MFIFLLFLTLTSGSDLDRCTTFDDVQAPNYTQHTSSMRGVYYPDEI...,2,10,FIFLLFLTL,0.621,0.042,1.148,-7.467,5.569763,0.116335,-0.061116,33.205116,0
3,AAU93319,MFIFLLFLTLTSGSDLDRCTTFDDVQAPNYTQHTSSMRGVYYPDEI...,6,20,LFLTLTSGSDLDRCT,1.021,0.230,1.049,0.927,5.569763,0.116335,-0.061116,33.205116,0
4,AAU93319,MFIFLLFLTLTSGSDLDRCTTFDDVQAPNYTQHTSSMRGVYYPDEI...,9,25,TLTSGSDLDRCTTFDDV,1.089,0.627,1.015,3.165,5.569763,0.116335,-0.061116,33.205116,0


In [6]:
from sklearn.metrics import roc_auc_score

best_models = ['et','catboost','xgboost','lightgbm','rf']

df_Results = pd.DataFrame(columns=['Classification', 'Dataset', 'Model', 'AUC'])

for m in best_models:
    
    print('-----------------------------------------------------')
    print('[START] - Processing model: ', m)
    print('-----------------------------------------------------')
    
    mo = create_model(m)
    
    print('-----------------------------------------------------')
    print('[START] - Tunning model: ', m)
    print('-----------------------------------------------------')
    
    tu = tune_model(mo)
    
    print('-----------------------------------------------------')
    print('[START] - Ensemble model: ', m)
    print('-----------------------------------------------------')
    
    en = ensemble_model(mo)
    
    mo_pred = predict_model(mo, test)
    mo_pred.dropna(inplace=True)
    
    tu_pred = predict_model(tu, test)
    tu_pred.dropna(inplace=True)
    
    en_pred = predict_model(en, test)
    en_pred.dropna(inplace=True)
    
    try:
        df_Results.loc[len(df_Results)] = [m, 'Valid', 'Model', roc_auc_score(mo_pred['Label'], mo_pred['target'])]
    except:
        df_Results.loc[len(df_Results)] = [m, 'Valid', 'Model', 'NA']
    try:
        df_Results.loc[len(df_Results)] = [m, 'Valid', 'Tunned', roc_auc_score(tu_pred['Label'], tu_pred['target'])]
    except:
        df_Results.loc[len(df_Results)] = [m, 'Valid', 'Tunned', 'NA']
    try:
        df_Results.loc[len(df_Results)] = [m, 'Valid', 'Ensembled', roc_auc_score(en_pred['Label'], en_pred['target'])]
    except:
        df_Results.loc[len(df_Results)] = [m, 'Valid', 'Ensembled', 'NA']
    
    mo_pred_sars = predict_model(mo, sars)
    mo_pred_sars.dropna(inplace=True)
    
    tu_pred_sars = predict_model(tu, sars)
    tu_pred_sars.dropna(inplace=True)
    
    en_pred_sars = predict_model(en, sars)
    en_pred_sars.dropna(inplace=True)
    
    try:
        df_Results.loc[len(df_Results)] = [m, 'SARS', 'Model', roc_auc_score(mo_pred_sars['Label'], mo_pred_sars['target'])]
    except:
        df_Results.loc[len(df_Results)] = [m, 'SARS', 'Model', 'NA']
    try:
        df_Results.loc[len(df_Results)] = [m, 'SARS', 'Tunned', roc_auc_score(tu_pred_sars['Label'], tu_pred_sars['target'])]
    except:
        df_Results.loc[len(df_Results)] = [m, 'SARS', 'Tunned', 'NA']
    try:
        df_Results.loc[len(df_Results)] = [m, 'SARS', 'Ensembled', roc_auc_score(en_pred_sars['Label'], en_pred_sars['target'])]
    except:
        df_Results.loc[len(df_Results)] = [m, 'SARS', 'Ensembled', 'NA']
    
    print('-----------------------------------------------------')
    print('[FINISHED] - Model: ', m)
    print('-----------------------------------------------------')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8642,0.9195,0.6139,0.8413,0.7098,0.6240,0.6371
1,0.8495,0.8844,0.5946,0.7979,0.6814,0.5857,0.5965
2,0.8328,0.9012,0.5598,0.7592,0.6444,0.5384,0.5491
3,0.8213,0.8756,0.5714,0.7115,0.6338,0.5175,0.5230
4,0.8464,0.8929,0.6023,0.7800,0.6797,0.5809,0.5893
5,0.8516,0.9053,0.6371,0.7746,0.6992,0.6019,0.6070
6,0.8504,0.9035,0.6062,0.7929,0.6871,0.5911,0.6003
7,0.8556,0.9034,0.6332,0.7923,0.7039,0.6100,0.6167
8,0.8337,0.8856,0.5598,0.7632,0.6459,0.5405,0.5516
9,0.8253,0.8743,0.5792,0.7212,0.6424,0.5286,0.5342


-----------------------------------------------------
[FINISHED] - Model:  rf
-----------------------------------------------------


In [7]:
print(df_Results.sort_values(by=['Dataset', 'AUC'], ascending=False))

   Classification Dataset      Model       AUC
1              et   Valid     Tunned        NA
19       lightgbm   Valid     Tunned  0.861111
18       lightgbm   Valid      Model  0.841176
6        catboost   Valid      Model  0.834096
8        catboost   Valid  Ensembled  0.834096
20       lightgbm   Valid  Ensembled  0.834096
7        catboost   Valid     Tunned  0.809091
14        xgboost   Valid  Ensembled  0.809091
24             rf   Valid      Model  0.790618
26             rf   Valid  Ensembled  0.785714
2              et   Valid  Ensembled  0.761364
12        xgboost   Valid      Model  0.742563
13        xgboost   Valid     Tunned  0.742563
0              et   Valid      Model  0.738095
25             rf   Valid     Tunned  0.738095
4              et    SARS     Tunned        NA
27             rf    SARS      Model  0.548562
23       lightgbm    SARS  Ensembled  0.537886
16        xgboost    SARS     Tunned  0.533125
21       lightgbm    SARS      Model  0.520833
28           

In [8]:
rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8485,0.8886,0.5869,0.8000,0.6771,0.5811,0.5930
1,0.8443,0.8706,0.5830,0.7865,0.6696,0.5707,0.5816
2,0.8474,0.8849,0.5869,0.7958,0.6756,0.5788,0.5903
3,0.8307,0.8568,0.5985,0.7277,0.6568,0.5458,0.5504
4,0.8380,0.8902,0.5598,0.7796,0.6517,0.5498,0.5626
5,0.8474,0.8883,0.6178,0.7729,0.6867,0.5875,0.5940
6,0.8358,0.8860,0.5521,0.7772,0.6456,0.5427,0.5561
7,0.8379,0.8695,0.5676,0.7737,0.6548,0.5521,0.5634
8,0.8274,0.8656,0.5598,0.7398,0.6374,0.5269,0.5358
9,0.8295,0.8640,0.5869,0.7308,0.6510,0.5399,0.5456


In [9]:
covid = pd.read_csv('795813_1365687_bundle_archive/input_covid.csv')

covid.head()

,parent_protein_id,protein_seq,start_position,end_position,peptide_seq,chou_fasman,emini,kolaskar_tongaonkar,parker,isoelectric_point,aromaticity,hydrophobicity,stability
0,6VYB_A,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...,1,5,MGILP,0.948,0.280,1.033,-2.72,6.03595,0.10929,-0.138642,31.377603
1,6VYB_A,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...,2,6,GILPS,1.114,0.379,1.070,-0.58,6.03595,0.10929,-0.138642,31.377603
2,6VYB_A,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...,3,7,ILPSP,1.106,0.592,1.108,-1.30,6.03595,0.10929,-0.138642,31.377603
3,6VYB_A,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...,4,8,LPSPG,1.324,0.836,1.053,1.44,6.03595,0.10929,-0.138642,31.377603
4,6VYB_A,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...,5,9,PSPGM,1.326,1.004,0.968,2.44,6.03595,0.10929,-0.138642,31.377603


In [11]:
pred_covid = predict_model(rf, covid)
pred_covid

,parent_protein_id,protein_seq,start_position,end_position,peptide_seq,chou_fasman,emini,kolaskar_tongaonkar,parker,isoelectric_point,aromaticity,hydrophobicity,stability,Label,Score
0,6VYB_A,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...,1,5,MGILP,0.948,0.280,1.033,-2.720,6.03595,0.10929,-0.138642,31.377603,0,0.3
1,6VYB_A,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...,2,6,GILPS,1.114,0.379,1.070,-0.580,6.03595,0.10929,-0.138642,31.377603,0,0.3
2,6VYB_A,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...,3,7,ILPSP,1.106,0.592,1.108,-1.300,6.03595,0.10929,-0.138642,31.377603,0,0.4
3,6VYB_A,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...,4,8,LPSPG,1.324,0.836,1.053,1.440,6.03595,0.10929,-0.138642,31.377603,0,0.5
4,6VYB_A,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...,5,9,PSPGM,1.326,1.004,0.968,2.440,6.03595,0.10929,-0.138642,31.377603,1,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20307,6VYB_A,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...,1258,1277,YVRKDGEWVLLSTFLGHHHH,0.947,0.406,1.063,0.155,6.03595,0.10929,-0.138642,31.377603,1,0.7
20308,6VYB_A,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...,1259,1278,VRKDGEWVLLSTFLGHHHHH,0.937,0.353,1.061,0.355,6.03595,0.10929,-0.138642,31.377603,1,0.6
20309,6VYB_A,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...,1260,1279,RKDGEWVLLSTFLGHHHHHH,0.960,0.647,1.047,0.645,6.03595,0.10929,-0.138642,31.377603,1,0.7
20310,6VYB_A,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...,1261,1280,KDGEWVLLSTFLGHHHHHHH,0.960,0.449,1.058,0.540,6.03595,0.10929,-0.138642,31.377603,1,0.6


In [12]:
pred_covid.to_csv('epitope-prediction-v3-covid.csv', index=False)

In [14]:
pred_covid['Label'].value_counts()/len(pred_covid)*100

1    58.970067
0    41.029933
Name: Label, dtype: float64